In [2]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math

In [3]:
start = time.time()
# import nodes
road_node = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\nodes.shp')
road_node = road_node.set_index('osmid')
road_nodes = road_node.to_crs(4326)
road_nodes['geometry_m'] = gpd.GeoSeries(road_nodes['geometry'], crs = 4326).to_crs(3043)
road_nodes

,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m
osmid,,,,,,,,,
103237949,4.431781e+06,503033.785157,3,-74.964440,40.036252,None,None,POINT (-74.96444 40.03625),POINT (-5689489.546 8448457.709)
103353127,4.431724e+06,502937.820011,3,-74.965565,40.035733,None,None,POINT (-74.96557 40.03573),POINT (-5689617.107 8448567.833)
103353090,4.431839e+06,503124.706820,3,-74.963374,40.036770,None,None,POINT (-74.96337 40.03677),POINT (-5689364.441 8448354.775)
103237976,4.431638e+06,503122.315306,4,-74.963403,40.034959,None,None,POINT (-74.96340 40.03496),POINT (-5689653.040 8448263.511)
103238007,4.431510e+06,503203.675365,4,-74.962450,40.033810,None,None,POINT (-74.96245 40.03381),POINT (-5689797.056 8448087.089)
...,...,...,...,...,...,...,...,...,...
9703813717,4.416438e+06,483689.901846,3,-75.190791,39.897861,None,None,POINT (-75.19079 39.89786),POINT (-5720575.973 8468970.364)
9703813743,4.416385e+06,483760.410069,3,-75.189965,39.897385,None,None,POINT (-75.18996 39.89738),POINT (-5720619.056 8468844.195)
9703813748,4.416433e+06,484049.808886,3,-75.186581,39.897820,None,None,POINT (-75.18658 39.89782),POINT (-5720414.758 8468450.854)


In [4]:
# import edges
road_edge = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\edges.shp')
road_edge = road_edge.set_index(['u','v','key'])
road_edges = road_edge.to_crs(4326)
road_edges['geometry_m'] = gpd.GeoSeries(road_edges['geometry'], crs = 4326).to_crs(3043)
road_edges

#Get both ways
RE = road_edges.reset_index()
RE2 = RE[['osmid','from','to','name','length','geometry']]
RE2.columns = ['osmid','to','from','name','length','geometry']
RE2 = RE2[['osmid','from','to','name','length','geometry']]
road_connections = pd.concat([RE, RE2]).to_crs(4326)
road_connections['key'] = road_connections['from'].astype(str) + '-' + road_connections['to'].astype(str)

In [5]:
# Get the graph object
graph = ox.graph_from_place('Philadelphia, United States', network_type="all", buffer_dist = 2000)
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [6]:
# Import greenspace and population grids
Penn_bound = gpd.read_file(r'C:\Users\bartb\Downloads\Pennsylvania\boundaries.shp')
Philly_greenspace = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\Public_greenspace.gpkg')
Philly_popgrid = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\CPoPGrid.gpkg')
Philly_bound = Penn_bound[Penn_bound['municipal1'] == 'PHILADELPHIA']

In [7]:
# Set crs to WGS84
Philly_bound = Philly_bound.to_crs(4326)
Philly_greenspace = Philly_greenspace.to_crs(4326)
Philly_popgrid = Philly_popgrid.to_crs(4326)

In [8]:
# Make sure only complete overlays progress and take their centroids
popgrid = Philly_popgrid.overlay(Philly_bound)
popgrid['centroid'] = popgrid.to_crs(4326).centroid

# CRS 3043 measures the world in metres.
popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)

# Only include grids that are within 99 % of the max area (they differ slightly with lon-lat)
popgrid['area'] = popgrid.area / popgrid.area.max()
popgrid = popgrid[(popgrid['area'] >= 0.99) & popgrid['PoP2015_Number'] > 0]

# Get lon and lat for performance calculation later
popgrid['grid_lon'] = popgrid['centroid_m'].x
popgrid['grid_lat'] = popgrid['centroid_m'].y
popgrid = popgrid.reset_index()
popgrid

C:\Users\bartb\AppData\Local\Temp\ipykernel_10524\1366358376.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['centroid'] = popgrid.to_crs(4326).centroid
C:\Users\bartb\AppData\Local\Temp\ipykernel_10524\1366358376.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['area'] = popgrid.area / popgrid.area.max()


,index,grid_id,PoP2015_Number,class_of_m,county,fed_aid_ur,fed_id_num,fips_area,fips_count,fips_mun_c,...,gpid,mslink,municipal,municipal1,geometry,centroid,centroid_m,area,grid_lon,grid_lat
0,6,259,113,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),0.996208,-5.675978e+06,8.459813e+06
1,7,260,135,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),0.996208,-5.675889e+06,8.459539e+06
2,13,400,137,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),0.996234,-5.676341e+06,8.459996e+06
3,16,403,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",POINT (-75.01115 40.13262),POINT (-5676072.561 8459176.309),0.996234,-5.676073e+06,8.459176e+06
4,17,404,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",POINT (-75.00890 40.13262),POINT (-5675982.949 8458902.986),0.996234,-5.675983e+06,8.458903e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,10223,20883,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",POINT (-75.25819 39.88221),POINT (-5725754.488 8476450.010),0.999924,-5.725754e+06,8.476450e+06
3887,10225,20885,39,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",POINT (-75.25369 39.88221),POINT (-5725576.204 8475897.317),0.999924,-5.725576e+06,8.475897e+06
3888,10226,20886,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",POINT (-75.25145 39.88221),POINT (-5725487.039 8475620.982),0.999924,-5.725487e+06,8.475621e+06
3889,10275,21024,3,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",POINT (-75.26043 39.88048),POINT (-5726119.978 8476637.252),0.999949,-5.726120e+06,8.476637e+06


In [9]:
# Get a 25 metre buffer and exclude the park area itself.
green_buffer = gpd.GeoDataFrame(geometry = Philly_greenspace.to_crs(3043).buffer(25).to_crs(4326))
Philly_greenspace['geometry_w_buffer'] = green_buffer
Philly_greenspace['geometry_w_buffer'] = gpd.GeoSeries(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])

C:\Users\bartb\AppData\Local\Temp\ipykernel_10524\3238172651.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])


In [10]:
# This function group components in itself that overlap (with the buffer set of 25 metres)
# https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
W = libpysal.weights.fuzzy_contiguity(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['components'] = W.component_labels
Philly_parks = Philly_greenspace.dissolve('components')

# Exclude parks below 0.04 ha.
Philly_parks = Philly_parks[Philly_parks.to_crs(3043).area > 400]

# Get the Philadelphia boundary buffer of 1000m to ensure equal grid oppertunities for park access
Bound_buffer = gpd.GeoDataFrame(
    geometry = np.repeat(Philly_bound.to_crs(3043).
                         buffer(1000).to_crs(4326),len(Philly_parks)), 
    crs = 4326).reset_index().iloc[:,1]
Bound_buffer.index = Philly_parks.index
Philly_parks['bound_buffer'] = Bound_buffer
Philly_parks

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 747 disconnected components.
 There are 585 islands with ids: 11, 15, 26, 28, 45, 52, 53, 61, 65, 68, 71, 89, 90, 91, 95, 96, 97, 98, 103, 104, 105, 106, 107, 108, 110, 117, 118, 127, 133, 134, 138, 139, 148, 149, 153, 154, 155, 156, 157, 159, 163, 168, 171, 173, 174, 177, 181, 182, 185, 186, 189, 190, 191, 192, 193, 195, 197, 198, 199, 200, 201, 202, 203, 217, 221, 228, 232, 235, 239, 240, 241, 242, 245, 249, 255, 256, 261, 262, 264, 266, 267, 272, 273, 275, 276, 280, 285, 286, 288, 290, 292, 301, 303, 304, 307, 310, 313, 315, 316, 318, 320, 321, 322, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 341, 342, 343, 344, 348, 349, 351, 355, 356, 357, 358, 359, 362, 363, 370, 371, 385, 437, 445, 449, 450, 451, 460, 461, 462, 463, 466, 467, 473, 483, 484, 485, 497, 498, 499, 503, 504, 547, 562, 564, 566, 569, 578, 579, 581, 582, 583, 5

,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,bound_buffer
components,,,,,,,,,,,,,,,,,,,,,
0,"POLYGON ((-75.09099 39.91406, -75.09099 39.914...",74457,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,4153.613200,2.281885e+05,"POLYGON ((-75.09191 39.91345, -75.09192 39.913...","POLYGON ((-75.09192 39.91345, -75.09193 39.913...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
1,"MULTIPOLYGON (((-75.02442 39.89516, -75.02516 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.02347 39.89392, -75.02348 39.893...","POLYGON ((-75.02348 39.89391, -75.02350 39.893...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
2,"MULTIPOLYGON (((-75.09305 39.93204, -75.09310 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.08855 39.93442, -75.08855 39.934...","POLYGON ((-75.08855 39.93442, -75.08859 39.934...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
3,"MULTIPOLYGON (((-75.06332 39.87934, -75.06355 ...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07984 39.88138, -75.07984 39.881...","POLYGON ((-75.07984 39.88137, -75.07983 39.881...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
4,"POLYGON ((-75.07444 39.89581, -75.07452 39.895...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07434 39.89568, -75.07442 39.895...","POLYGON ((-75.07442 39.89565, -75.07444 39.895...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,"POLYGON ((-75.18593 40.13635, -75.18627 40.135...",242041,Fee,Fee,LOC,CITY,Upper Dublin Township,LOC,CITY,Upper Dublin Township,...,2020,None,None,None,None,1490.477107,6.204571e+04,"POLYGON ((-75.18593 40.13650, -75.18592 40.136...","POLYGON ((-75.18592 40.13650, -75.18590 40.136...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
743,"POLYGON ((-75.26429 40.09641, -75.26362 40.096...",244970,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,2020,None,None,None,None,1275.860609,8.335866e+04,"POLYGON ((-75.26442 40.09652, -75.26440 40.096...","POLYGON ((-75.26440 40.09653, -75.26438 40.096...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
744,"POLYGON ((-75.20583 40.11865, -75.20772 40.116...",244971,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,2020,None,None,None,None,756.792370,2.828873e+04,"POLYGON ((-75.20571 40.11877, -75.20569 40.118...","POLYGON ((-75.20569 40.11876, -75.20568 40.118...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."


In [11]:
# Exclide the park area from the park+buffer area
parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
parks_within_range = parks_within_range.loc[:, ~Philly_parks.columns.isin(['bound_buffer'])]
parks_within_range = parks_within_range.reset_index()

# Get park centroids and park buffers as variables itself.
parks_within_range['centroid'] = parks_within_range.centroid
parks_within_range['centroid_m'] = parks_within_range['centroid'].to_crs(3043)
parks_within_range['geometry_w_buffer'] = gpd.GeoSeries(parks_within_range['geometry_w_buffer'], crs = 4326)
parks_within_range['geometry_w_buffer_m'] = parks_within_range['geometry_w_buffer'].to_crs(3043)
parks_within_range = gpd.GeoDataFrame(parks_within_range, geometry = 'geom buffer diff', crs = 4326)
parks_within_range

C:\Users\bartb\AppData\Local\Temp\ipykernel_10524\3404927833.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
C:\Users\bartb\AppData\Local\Temp\ipykernel_10524\3404927833.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range['centroid'] = parks_within_range.centroid


,components,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,...,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,centroid,centroid_m,geometry_w_buffer_m
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,...,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...",POINT (-75.25165 40.05653),POINT (-5697693.589 8484600.730),"POLYGON ((-5697129.449 8485382.576, -5697129.4..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...",POINT (-75.24714 39.91942),POINT (-5719375.750 8477008.358),"POLYGON ((-5719780.450 8476846.530, -5719785.6..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...",POINT (-75.25640 39.96241),POINT (-5712881.953 8480355.991),"POLYGON ((-5719127.059 8477242.926, -5719138.2..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...",POINT (-75.15001 39.96195),POINT (-5708732.995 8467282.252),"POLYGON ((-5708763.625 8467321.486, -5708762.4..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...",POINT (-75.17237 40.03387),POINT (-5698171.846 8473736.251),"POLYGON ((-5698218.752 8473779.070, -5698217.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...",POINT (-75.25763 39.94953),POINT (-5714985.845 8479845.686),"POLYGON ((-5715014.449 8479809.842, -5715014.8..."
470,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...",POINT (-75.27612 39.96913),POINT (-5712590.186 8483121.334),"POLYGON ((-5712589.472 8483026.846, -5712589.5..."
471,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25664 39.94598, -75.25669 39.945...","POLYGON ((-75.25669 39.94597, -75.25671 39.945...",POINT (-75.25609 39.94665),POINT (-5715384.150 8479507.849),"POLYGON ((-5715513.534 8479540.584, -5715516.4..."
472,739,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,...,None,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27789 39.96685, -75.27788 39.966...",POINT (-75.27714 39.96644),POINT (-5713059.443 8483108.295),"POLYGON ((-5713024.922 8483221.016, -5713022.5..."


In [12]:
start_time = time.time()
ParkRoad = pd.DataFrame()
mat = list()
# Get the nodes within the park buffer
for i in range(len(parks_within_range)):
    dist = road_nodes['geometry'].to_crs(3043).distance(parks_within_range['geometry'].to_crs(
        3043)[i])
    buf_nodes = road_nodes[(dist < 25) & (dist > 0)]
    mat.append(list(np.repeat(i, len(buf_nodes))))
    ParkRoad = pd.concat([ParkRoad, buf_nodes])
    if i % 10 == 0: print(round(i/len(parks_within_range)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')

# Park no list conversion
mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

# Format
ParkRoad['Park_No'] = mat_u
ParkRoad = ParkRoad.reset_index()
ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
print('100 % done',round((time.time() - start_time) / 60,2),' mns')
ParkRoad

    

0.0 % done 0.01  mns
2.1 % done 0.23  mns
4.2 % done 0.32  mns
6.3 % done 0.57  mns
8.4 % done 0.87  mns
10.5 % done 1.03  mns
12.7 % done 1.26  mns
14.8 % done 1.59  mns
16.9 % done 1.66  mns
19.0 % done 1.72  mns
21.1 % done 1.79  mns
23.2 % done 1.86  mns
25.3 % done 1.93  mns
27.4 % done 2.0  mns
29.5 % done 2.07  mns
31.6 % done 2.14  mns
33.8 % done 2.2  mns
35.9 % done 2.29  mns
38.0 % done 2.37  mns
40.1 % done 2.44  mns
42.2 % done 2.52  mns
44.3 % done 2.58  mns
46.4 % done 2.65  mns
48.5 % done 2.72  mns
50.6 % done 2.79  mns
52.7 % done 2.85  mns
54.9 % done 2.9  mns
57.0 % done 2.96  mns
59.1 % done 3.02  mns
61.2 % done 3.09  mns
63.3 % done 3.16  mns
65.4 % done 3.24  mns
67.5 % done 3.3  mns
69.6 % done 3.37  mns
71.7 % done 3.45  mns
73.8 % done 3.53  mns
75.9 % done 3.6  mns
78.1 % done 3.68  mns
80.2 % done 3.74  mns
82.3 % done 3.83  mns
84.4 % done 3.89  mns
86.5 % done 3.97  mns
88.6 % done 4.05  mns
90.7 % done 4.13  mns
92.8 % done 4.22  mns
94.9 % done 4.3  mns

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,8.483120e+06
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,8.482975e+06
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),0,-5.697523e+06,8.483422e+06
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,8.482831e+06
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,8.482767e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),472,-5.713031e+06,8.483035e+06
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),473,-5.713613e+06,8.482370e+06
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),473,-5.713713e+06,8.482492e+06
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),473,-5.713536e+06,8.482556e+06


In [13]:
# Save as file
ParkRoad[['Park_No','geometry']].to_crs(4326).to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [14]:
# Check the road nodes to compare visually (QGIS) of a certain park
ParkRoad[ParkRoad['Park_No'] == 453]

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
7224,109890052,4.429296e+06,493169.700669,3,-75.080034,40.013838,None,None,POINT (-75.08003 40.01384),POINT (-5697684.279 8461406.828),453,-5.697684e+06,8.461407e+06
7225,110188472,4.429317e+06,493092.570321,3,-75.080938,40.014029,None,None,POINT (-75.08094 40.01403),POINT (-5697690.013 8461527.279),453,-5.697690e+06,8.461527e+06
7226,110291816,4.429273e+06,493150.989575,4,-75.080253,40.013627,None,None,POINT (-75.08025 40.01363),POINT (-5697726.578 8461422.644),453,-5.697727e+06,8.461423e+06


In [15]:
# Merge park info with park entry point info.
PtR = pd.merge(ParkRoad, parks_within_range[['geometry','geometry_w_buffer']], left_on = 'Park_No', right_index = True)

# Get a 500m buffer from the closest park entry point 
# (most people won't walk any further than 500m, extra gravity is negligible)
PtR['park_size_walkable'] = PtR['geometry_m'].buffer(500).to_crs(4326).intersection(PtR['geometry_y'])
PtR['walk_area'] = PtR['park_size_walkable'].to_crs(3043).area
PtR['park_area'] = PtR['geometry_y'].to_crs(3043).area
PtR['share_walked'] = PtR['walk_area'] / PtR['park_area']

# Get the normal and logarithmic inflation factors compared with its mean.
PtR['size_infl_factor'] = PtR['walk_area'] / PtR['walk_area'].mean()
PtR['log_walk'] = PtR['walk_area'].apply(math.log)
PtR['log_infl_factor'] = (PtR['log_walk'] - (PtR['log_walk'].min()-1)) / (PtR['log_walk'] - (PtR['log_walk'].min()-1)).mean()
PtR

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_x,geometry_m,...,park_lat,geometry_y,geometry_w_buffer,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,log_walk,log_infl_factor
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),...,8.483120e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.504099,12.199668,1.157274
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),...,8.482975e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.205875,12.582598,1.219697
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),...,8.483422e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","MULTIPOLYGON (((-75.24062 40.05796, -75.24030 ...",440724.683283,4.192678e+06,0.105118,3.335766,12.996176,1.287115
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),...,8.482831e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061025,4.192678e+06,0.058450,1.854846,12.409275,1.191443
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),...,8.482767e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.419240,12.141595,1.147808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),...,8.483035e+06,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...","POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.149369,9.890135,0.780792
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),...,8.482370e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,10.761647,0.922859
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),...,8.482492e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,10.761647,0.922859
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),...,8.482556e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069,10.761647,0.922859


In [16]:
# Check the logarithmic values.
log_area = gpd.GeoDataFrame(PtR['park_size_walkable'], geometry = 'park_size_walkable', crs = 4326).to_crs(3043).area.apply(
    math.log)
(log_area - (log_area.min()-1))

0       7.099296
1       7.482227
2       7.895804
3       7.308904
4       7.041224
          ...   
7341    4.789764
7342    5.661276
7343    5.661276
7344    5.661276
7345    5.661276
Length: 7346, dtype: float64

In [123]:
# Save the polygons of walkable areas from certain access points
gpd.GeoDataFrame(PtR[['Park_No','park_size_walkable']], geometry = 'park_size_walkable', crs = 4326
                ).to_file(r'C:\Users\bartb\Downloads\parks.shp')

In [18]:
# Check all parks within 1000m radius
start_time = time.time()
len1 = len(popgrid)
len2 = len(ParkRoad)
len3 = int(np.ceil(len(ParkRoad)/1000))
output = pd.DataFrame()
len_mat = 0
# Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
for i in range(len3):
    # Check all grid-park combinations per 1000
    l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
    listed = pd.DataFrame(list(product(l1, l2)))
    
    # Merge grid and park information
    grid_merged = pd.merge(listed, 
                           popgrid[['grid_id','centroid','centroid_m','grid_lon','grid_lat']],
                           left_on = 0, right_index = True)
    node_merged = pd.merge(grid_merged, 
                           PtR[['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                               'log_infl_factor','share_walked','park_area','walk_area']], 
                           left_on = 1, right_index = True)
    
    # Preset index for merging
    len4 = len(node_merged)
    node_merged['key'] = range(0,len(node_merged))
    node_merged = node_merged.set_index('key')
    node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]
        
    # Create lists for better computational performance
    glon = list(node_merged['grid_lon'])
    glat = list(node_merged['grid_lat'])
    plon = list(node_merged['park_lon'])
    plat = list(node_merged['park_lat'])
    infl = list(node_merged['log_infl_factor'])
    
    # Get the plain and inflated euclidean distances
    mat = np.repeat(None,len(node_merged))
    mat2 = np.repeat(None,len(node_merged))
    for j in range(len(node_merged)):
        mat[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl[j]
        mat2[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2)
    
    # Check if distances are within the log-adjusted 1000m and join remaining info and concat in master df per 1000.
    mat_df = pd.DataFrame(mat)
    mat_df = mat_df[mat_df[0] <=1000].dropna()
    mat_df = pd.DataFrame(mat_df)
    mat_df = pd.merge(mat_df, pd.DataFrame(mat2), left_index = True, right_index = True, how = 'left')
    mat_df.columns = ['adjusted euclidean','raw euclidean']    
    mat_df = mat_df.join(node_merged)
        
    output = pd.concat([output, mat_df])

    print((i+1),'/',len3,'done',round((time.time() - start_time) / 60,2),' mns')
    print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within Gravity adjusted 1000m:',len(mat_df))
    
    # Checks the number of the parks within 1000m.
    len_mat = len_mat + len(mat_df)
    
# Renaming columns
print('total combinations within distance',len_mat)
output.columns = ['adjusted euclidean','raw euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'grid_lon','grid_lat',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                  'park_lon','park_lat','log_infl_factor','parkshare_walked','park_area','walk_area']

output = output[['raw euclidean','log_infl_factor','adjusted euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','walk_area']]
output

1 / 8 done 0.22  mns
of 3891000 within Gravity adjusted 1000m: 18478
2 / 8 done 0.45  mns
of 3891000 within Gravity adjusted 1000m: 20377
3 / 8 done 0.69  mns
of 3891000 within Gravity adjusted 1000m: 19113
4 / 8 done 0.92  mns
of 3891000 within Gravity adjusted 1000m: 16071
5 / 8 done 1.15  mns
of 3891000 within Gravity adjusted 1000m: 13020
6 / 8 done 1.38  mns
of 3891000 within Gravity adjusted 1000m: 15006
7 / 8 done 1.62  mns
of 3891000 within Gravity adjusted 1000m: 15368
8 / 8 done 1.72  mns
of 346000 within Gravity adjusted 1000m: 3899
total combinations within distance 121332


,raw euclidean,log_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area
913,1089.848567,1.157274,941.737382,913,0,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
914,1067.304885,1.157274,922.257403,914,0,6265,POINT (-75.24022 40.06019),POINT (-5696662.252 8483389.707),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
915,1121.233555,1.157274,968.857129,915,0,6266,POINT (-75.23797 40.06019),POINT (-5696573.814 8483114.956),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
956,779.289461,1.157274,673.383477,956,0,6407,POINT (-75.24022 40.05846),POINT (-5696937.014 8483301.273),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
957,851.662627,1.157274,735.921079,957,0,6408,POINT (-75.23797 40.05846),POINT (-5696848.569 8483026.507),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177802,1014.348951,1.039963,975.369994,2720,7302,12225,POINT (-75.24920 39.98767),POINT (-5708569.997 8480770.261),464,111706635,POINT (-75.24989 39.99370),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707635.357 8481164.408),96763.256115
1181623,1001.459054,1.014538,987.108923,2650,7303,11944,POINT (-75.24247 39.99113),POINT (-5707753.078 8480121.541),464,3618217765,POINT (-75.24980 39.99311),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707727.165 8481122.665),82788.847320
1181657,903.044102,1.014538,890.10418,2684,7303,12085,POINT (-75.24471 39.98940),POINT (-5708117.175 8480308.183),464,3618217765,POINT (-75.24980 39.99311),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707727.165 8481122.665),82788.847320
1181692,934.702623,1.014538,921.309059,2719,7303,12224,POINT (-75.25145 39.98767),POINT (-5708658.692 8481045.677),464,3618217765,POINT (-75.24980 39.99311),"POLYGON ((-75.24989 39.99303, -75.25100 39.992...",POINT (-5707727.165 8481122.665),82788.847320


In [ ]:
# Save the park entry centroids with adjusted euclidean distances
park_entry_centroid = gpd.GeoDataFrame(output[['adjusted euclidean','Grid_No','Park_No','Parkroad_osmid',
                                               'Parkroad_coords_centroid']],geometry = 'Parkroad_coords_centroid', crs = 4326)
park_entry_centroid.to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [20]:
# Check park and grid no with adjusted score (if the lowest is taken further on)
output[(output['Park_No'] == 231) & (output['Grid_No'] == 2639)]

,raw euclidean,log_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area
434540,214.025883,0.958677,223.25136,2639,5111,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809486,POINT (-75.11806 39.99394),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702367.869 8465026.271),58769.051358
438431,208.087815,0.958677,217.057335,2639,5112,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,5547581605,POINT (-75.11819 39.99395),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702370.518 8465042.292),58769.051358
442322,315.122553,0.958677,328.705751,2639,5113,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809494,POINT (-75.11790 39.99456),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702262.806 8465038.940),58769.051358
446213,456.479714,0.958677,476.156041,2639,5114,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809504,POINT (-75.11773 39.99542),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702119.247 8465062.415),58769.051358
450104,334.672172,0.958677,349.098047,2639,5115,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556737,POINT (-75.11788 39.99468),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702242.736 8465042.228),58769.051358
453995,480.640883,0.958677,501.358665,2639,5116,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109899816,POINT (-75.11966 39.99567),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702155.608 8465312.404),58769.051358
457886,257.779648,0.958677,268.89111,2639,5117,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109999351,POINT (-75.11999 39.99417),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702407.718 8465274.162),58769.051358
461777,202.093837,0.958677,210.804989,2639,5118,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556736,POINT (-75.11909 39.99406),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702389.554 8465157.699),58769.051358


In [21]:
# Check park and another (furter away) grid no with adjusted score
pd.DataFrame(output[(output['Park_No'] == 231) & (output['Grid_No'] == 2500)])


,raw euclidean,log_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area
442183,881.601233,0.958677,919.602208,2500,5113,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109809494,POINT (-75.11790 39.99456),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702262.806 8465038.940),58769.051358
446074,744.171919,0.958677,776.249073,2500,5114,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109809504,POINT (-75.11773 39.99542),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702119.247 8465062.415),58769.051358
449965,862.227068,0.958677,899.392929,2500,5115,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,2371556737,POINT (-75.11788 39.99468),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702242.736 8465042.228),58769.051358
453856,690.904593,0.958677,720.685686,2500,5116,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109899816,POINT (-75.11966 39.99567),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702155.608 8465312.404),58769.051358
457747,945.81404,0.958677,986.582875,2500,5117,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,109999351,POINT (-75.11999 39.99417),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702407.718 8465274.162),58769.051358
461638,955.209924,0.958677,996.383765,2500,5118,11289,POINT (-75.11895 39.99976),POINT (-5701475.942 8465436.514),231,2371556736,POINT (-75.11909 39.99406),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702389.554 8465157.699),58769.051358


In [22]:
start_time = time.time()

# Reinstate geographic elements
gp_entry = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
gp_entry['Grid_m_centroid'] = gpd.GeoSeries(gp_entry['Grid_m_centroid'], crs = 3043)
gp_entry['Parkroad_coords_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_coords_centroid'], crs = 4326)
gp_entry['Parkroad_m_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_m_centroid'], crs = 3043)

# Get index for merging later
roads = road_nodes.reset_index()

# Get the nearest entrance point for the grid centroids
mat2 = np.repeat(None,len(gp_entry))
for i in range(len(gp_entry)):
    nearest = int(roads['geometry'].sindex.nearest(gp_entry['Grid_coords_centroid'].iloc[i])[1])
    mat2[i] = roads['osmid'].iloc[nearest]
    if i % 100000 == 0: print(round(i/len(gp_entry)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')
print(len(mat2[mat2 != None]))

# Add all the information and the distance from the nearest point to the grid centroid in a geodataframe
pd.DataFrame(mat2)
gp_entry['grid_osm'] = mat2
gp_entry = pd.merge(gp_entry, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry['geometry_m'] = gp_entry['geometry'].to_crs(3043)
gp_entry['grid_entry_dist'] = round(gp_entry['Grid_m_centroid'].distance(gp_entry['geometry_m'].to_crs(3043)),3)
gp_entry = gp_entry.reset_index()
print('100 % done', round((time.time() - start_time) / 60,2),' mns')
gp_entry

0.0 % done 0.0  mns
82.4 % done 0.15  mns
121332
100 % done 0.19  mns


,index,raw euclidean,log_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area,grid_osm,geometry,geometry_m,grid_entry_dist
0,913,1089.848567,1.157274,941.737382,913,0,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,5550941629,POINT (-75.24218 40.06059),POINT (-5696675.232 8483650.229),76.774
1,4804,1000.053575,1.219697,819.919784,913,1,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991191,POINT (-75.23908 40.05480),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,5550941629,POINT (-75.24218 40.06059),POINT (-5696675.232 8483650.229),76.774
2,8695,809.602143,1.287115,629.005179,913,2,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,5550942021,POINT (-75.24251 40.05535),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697523.197 8483422.258),440724.683283,5550941629,POINT (-75.24218 40.06059),POINT (-5696675.232 8483650.229),76.774
3,12586,975.375476,1.191443,818.650529,913,3,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991199,POINT (-75.23750 40.05578),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697257.444 8482831.074),245064.061025,5550941629,POINT (-75.24218 40.06059),POINT (-5696675.232 8483650.229),76.774
4,16477,985.342204,1.147808,858.455749,913,4,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991204,POINT (-75.23678 40.05623),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697156.554 8482766.601),187511.352028,5550941629,POINT (-75.24218 40.06059),POINT (-5696675.232 8483650.229),76.774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121327,867366,346.669003,0.894541,387.538222,3564,7222,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,9454222002,POINT (-75.18418 39.93510),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714374.248 8470084.472),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045
121328,871257,332.31715,0.894541,371.494412,3564,7223,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,9454221987,POINT (-75.18411 39.93502),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714383.228 8470071.522),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045
121329,1001658,934.055458,0.976700,956.338569,1671,7257,8304,POINT (-75.12568 40.03602),POINT (-5695975.394 8468138.090),459,110370351,POINT (-75.11925 40.03343),"POLYGON ((-75.11763 40.03473, -75.11762 40.034...",POINT (-5696130.967 8467217.082),65639.417845,110217533,POINT (-75.12540 40.03578),POINT (-5696002.259 8468090.585),54.575
121330,1044208,818.453611,0.888711,920.944736,1420,7268,7683,POINT (-75.24471 40.04293),POINT (-5699587.531 8483054.833),461,111746732,POINT (-75.25087 40.04170),"POLYGON ((-75.25047 40.04210, -75.25081 40.041...",POINT (-5700026.381 8483745.686),38260.198018,5550942667,POINT (-75.24381 40.04170),POINT (-5699746.753 8482881.628),235.269


In [23]:
# Get only necessary road connections columns for network performance
road_conn = road_connections[['osmid','key','length','geometry']]
road_conn = road_conn.set_index('key')

In [57]:
# Check distances
gp_entry['Grid_m_centroid'].distance(gp_entry['Parkroad_m_centroid'].to_crs(3043)) / gp_entry['log_infl_factor']

0         941.737382
1         819.919784
2         629.005179
3         818.650529
4         858.455749
             ...    
121327    387.538222
121328    371.494412
121329    956.338569
121330    920.944736
121331    923.676701
Length: 121332, dtype: float64

In [25]:
# Shortest path algorithm

import warnings
warnings.filterwarnings("ignore")

start_time = time.time()

parknode = list(gp_entry['Parkroad_osmid'])
gridnode = list(gp_entry['grid_osm'])

s_mat = list([])
s_mat1 = list([])
s_mat2 = list([])
s_mat3 = list([])
s_mat4 = list([])

for i in range(len(gp_entry)):
    # First try from population grid to park Dijkstra algorithm
    try:
        shortest = nx.shortest_path(graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
        s_mat.append(shortest)
        shortest_to = list(shortest[1:len(shortest)])
        shortest_to.append(0)
        s_mat1.append(shortest_to)
        s_mat2.append(list(np.repeat(i, len(shortest))))
        s_mat3.append(list(np.arange(0, len(shortest))))
        s_mat4.append('normal way')
        
    except:
        try:
            # Check the reverse
            shortest = nx.shortest_path(graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('reverse way')
        except:
            try:
                # Get the route to the nearest park node
                new_node = pd.DataFrame((abs(gp_entry['Parkroad_coords_centroid'].x[123381] - PtR['geometry_y'].x)**2
                                         + abs(gp_entry['Parkroad_coords_centroid'].y[123381] - PtR['geometry_y'].y)**2
                                        )**(1/2)).join(PtR['osmid']).sort_values(0).iloc[1,1]
                shortest = nx.shortest_path(graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('nearest way')
            except:
                # Print if none of the above three options gives any result
                print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                pass
            
    if i % 10000 == 0: print(round(i/len(gp_entry)*100,2),'% done', round((time.time() - start_time) / 60,2),' mns')
print('100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')


0.0 % done 0.0  mns
8.24 % done 0.13  mns
16.48 % done 0.31  mns
24.73 % done 0.45  mns
32.97 % done 0.61  mns
41.21 % done 0.77  mns
49.45 % done 0.88  mns
57.69 % done 1.05  mns
65.93 % done 1.17  mns
74.18 % done 1.25  mns
82.42 % done 1.37  mns
90.66 % done 1.51  mns
98.9 % done 1.62  mns
100 % pathfinding done 1.63  mns


In [26]:
start_time = time.time()
# Unpack lists and add them in a dataframe
s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()

# Format df
routes.columns = ['from','to','route','step']
mat_key = list([])
for i in range(len(routes)):
    mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
routes['key'] = mat_key
routes = routes.set_index('key')

# Add route information
routes = routes.join(road_conn, how = 'left')
routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
routes.sort_values(by = ['route','step'])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes

time taken 0.66 mns


,from,to,route,step,osmid,length,geometry
key,,,,,,,
1015355888-0,1015355888,0,7418,20,NaN,NaN,None
1015355888-0,1015355888,0,7604,17,NaN,NaN,None
1015355888-0,1015355888,0,7837,14,NaN,NaN,None
1015355888-0,1015355888,0,8108,23,NaN,NaN,None
1015355888-0,1015355888,0,8771,10,NaN,NaN,None
...,...,...,...,...,...,...,...
989146009-2699409740,989146009,2699409740,69826,5,476000203,56.075,"LINESTRING (-75.18716 39.95470, -75.18705 39.9..."
989146009-2699409740,989146009,2699409740,69880,5,476000203,56.075,"LINESTRING (-75.18716 39.95470, -75.18705 39.9..."
989146009-2699409740,989146009,2699409740,69881,5,476000203,56.075,"LINESTRING (-75.18716 39.95470, -75.18705 39.9..."


In [27]:
# Add grid to road entry information to the line geometry and route.

start_time = time.time()
routegrid = list([])
for i in range(len(gp_entry)):
    routegrid.append(LineString([(gp_entry['geometry'][i]),(gp_entry['Grid_coords_centroid'][i])]))
gridline = gpd.GeoDataFrame(geometry = routegrid, crs = 4326)
gridline = gridline.join(gp_entry[['grid_entry_dist']])
gridline = gridline.reset_index()
gridline.columns = ['route','geometry','length']
gridline['grid_osm'] = '0-' + gp_entry['grid_osm'].astype(str)
gridline = gridline.set_index('grid_osm')
gridline['step'] = -1
gridline['osmid'] = np.nan
gridline['to'] = np.where(s_mat4 == 'normal way',gp_entry['Parkroad_osmid'],gp_entry['grid_osm'])

gridline['from'] = 0
gridline = gridline[['from','to','route','step','osmid','length','geometry']]
routes_stack = pd.concat([routes, gridline])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes_stack[routes_stack['route'] == 1].sort_values(by = 'step')

time taken 0.26 mns


,from,to,route,step,osmid,length,geometry
0-5550941629,0,5550941629,1,-1,NaN,76.774,"LINESTRING (-75.24218 40.06059, -75.24247 40.0..."
5550941629-5550941630,5550941629,5550941630,1,0,484741165,16.018,"LINESTRING (-75.24218 40.06059, -75.24231 40.0..."
5550941630-4774911885,5550941630,4774911885,1,1,484741165,226.414,"LINESTRING (-75.24231 40.06069, -75.24250 40.0..."
4774911885-4244000117,4774911885,4244000117,1,2,12187778,102.832,"LINESTRING (-75.24418 40.05970, -75.24343 40.0..."
4244000117-110443757,4244000117,110443757,1,3,12187778,232.152,"LINESTRING (-75.24336 40.05903, -75.24160 40.0..."
110443757-110305724,110443757,110305724,1,4,108593573,226.652,"LINESTRING (-75.24148 40.05751, -75.24177 40.0..."
110305724-5550942021,110305724,5550942021,1,5,375552996,117.099,"LINESTRING (-75.24344 40.05613, -75.24261 40.0..."
5550942021-109991185,5550942021,109991185,1,6,375552996,230.444,"LINESTRING (-75.24251 40.05535, -75.24217 40.0..."
109991185-109991191,109991185,109991191,1,7,375552995,175.298,"LINESTRING (-75.23908 40.05480, -75.24069 40.0..."
109991191-0,109991191,0,1,8,NaN,NaN,None


In [28]:
start_time = time.time()
# get single (dissolved) line per route, attach information.
routes2 = routes_stack[['route','geometry']].dissolve('route')
routes2['way_calculated'] = s_mat4
routes2['route_cost'] = routes.groupby('route')['length'].sum()
routes2['num_steps'] = routes.groupby('route')['step'].max()
routes2.index = routes2.index.astype(int)
routes2 = pd.merge(routes2, gp_entry[['Grid_No','Park_No',
                                      'Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm','walk_area','log_infl_factor']],
                   left_index = True, right_index = True)

# Get cost calculations
routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
routes2['grav_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['log_infl_factor']
routes2['parkcost_km2'] = routes2['grav_total_cost'] * (routes2['walk_area'] /1000000)
routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes2 

time taken 0.43 mns


,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area,log_infl_factor,raw_total_cost,grav_total_cost,parkcost_km2,gridpark_no
route,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1151.611,7,913,0,0,76.774,109991185,5550941629,198723.068185,1.157274,1228.385,1061.446617,210.933929,913-0
1,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,1326.909,8,913,0,1,76.774,109991191,5550941629,291442.444263,1.219697,1403.683,1150.845805,335.405314,913-0
2,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,921.167,6,913,0,2,76.774,5550942021,5550941629,440724.683283,1.287115,997.941,775.331516,341.707737,913-0
3,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,1499.505,9,913,0,3,76.774,109991199,5550941629,245064.061025,1.191443,1576.279,1322.999879,324.219723,913-0
4,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,1578.911,10,913,0,4,76.774,109991204,5550941629,187511.352028,1.147808,1655.685,1442.475823,270.480592,913-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121327,"MULTILINESTRING ((-75.18256 39.93482, -75.1826...",normal way,382.198,7,3564,452,7222,110.045,9454222002,110255941,39653.440881,0.894541,492.243,550.274110,21.820262,3564-452
121328,"MULTILINESTRING ((-75.18256 39.93482, -75.1826...",normal way,367.292,5,3564,452,7223,110.045,9454221987,110255941,39653.440881,0.894541,477.337,533.610824,21.159505,3564-452
121329,"MULTILINESTRING ((-75.11786 40.03327, -75.1177...",normal way,968.217,8,1671,459,7257,54.575,110370351,110217533,65639.417845,0.976700,1022.792,1047.192037,68.737076,1671-459


In [29]:
start_time = time.time()
# Get the entry point of the park that is closest to the grid centroid
gridpark_score = routes2.iloc[routes2.groupby('gridpark_no')['grav_total_cost'].idxmin()]

# Check if the route to the closest entry point is within 1000m.
gridpark_score = gridpark_score[gridpark_score['grav_total_cost'] <= 1000]

# The score is the inverse of the total cost (if a park is 100m (on route) away its score is 900 (1000 - 100))
gridpark_score['score'] = 1000 - gridpark_score['grav_total_cost']

# Get the population weighted score
gridpark_score = pd.merge(gridpark_score, popgrid[['PoP2015_Number','geometry']],
                          left_on = 'Grid_No', right_index = True, how = 'outer')


# Fill Null certain columns for processing reasons
gridpark_score['score'] = gridpark_score['score'].fillna(0).round()
gridpark_score['population_score'] = gridpark_score['score'] * gridpark_score['PoP2015_Number']

gridpark_score['Park_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score.index = gridpark_score.index.fillna(-1)
gridpark_score.index = gridpark_score.index.astype(int)

gridpark_score['num_steps'] = gridpark_score['num_steps'].fillna(-1)
gridpark_score['num_steps'] = gridpark_score['num_steps'].astype(int)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')

gridpark_score = gridpark_score.reset_index()

gridpark_score

time taken 0.01 mns


,index,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,...,walk_area,log_infl_factor,raw_total_cost,grav_total_cost,parkcost_km2,gridpark_no,score,PoP2015_Number,geometry_y,population_score
0,117767,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,649.034,5,10,349,349,47.215,1.100560e+08,...,96266.908663,1.039125,696.249,670.033970,64.502099,10-349,330.0,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",41910.0
1,57458,"MULTILINESTRING ((-74.99552 40.05713, -74.9964...",normal way,809.681,8,1000,35,35,20.765,1.102507e+08,...,241431.013780,1.189008,830.446,698.435827,168.624070,1000-35,302.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",56474.0
2,57473,"LINESTRING (-74.99992 40.05834, -74.99992 40.0...",normal way,0.000,0,1000,45,45,20.765,7.826689e+09,...,263123.707902,1.203034,20.765,17.260526,4.541654,1000-45,983.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",183821.0
3,56804,"MULTILINESTRING ((-74.99333 40.05669, -74.9945...",normal way,415.485,4,1001,35,35,58.594,1.104250e+08,...,281553.844925,1.214070,474.079,390.487419,109.943234,1001-35,610.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",34770.0
4,56821,"LINESTRING (-74.99565 40.05877, -74.99543 40.0...",normal way,0.000,0,1001,45,45,58.594,1.104250e+08,...,199688.560770,1.158064,58.594,50.596492,10.103541,1001-45,949.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",54093.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7002,-1,None,NaN,NaN,-1,3886,-1,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",0.0
7003,-1,None,NaN,NaN,-1,3887,-1,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,39,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",0.0
7004,-1,None,NaN,NaN,-1,3888,-1,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",0.0
7005,-1,None,NaN,NaN,-1,3889,-1,-1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",0.0


In [37]:
# Save the routes (lines) and grids itself with the score-information

gridpark_lines = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_x', crs = 4326)
gridpark_nodes = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_y', crs = 4326)

gridpark_lines[['geometry_x','grav_total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_lines.shp')

gridpark_nodes[['geometry_y','grav_total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_nodes.shp')

In [31]:
start_time = time.time()
# Group the park scores per grid.
grid_score = pd.DataFrame(gridpark_score.groupby('Grid_No')['score'].sum().sort_values(ascending=False))
grid_score['population_score (mln)'] = gridpark_score.groupby('Grid_No')['population_score'].sum()/1000000
grid_score['parkscore_km2'] = gridpark_score.groupby('Grid_No')['parkcost_km2'].sum()
grid_score['count'] = gridpark_score.groupby('Grid_No')['score'].count()

grid_score = grid_score.join(popgrid[['geometry','PoP2015_Number']])
grid_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Grid_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

# The merge contains grids with no access to parks within 1000m, these scores are set to 0
grid_score = gpd.GeoDataFrame(grid_score, geometry = 'geometry', crs = 4326)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
grid_score

time taken 0.02 mns


,score,population_score (mln),parkscore_km2,count,geometry,PoP2015_Number,lines
Grid_No,,,,,,,
3584,3277.0,0.888067,34.164910,5,"POLYGON ((-75.14477 39.93323, -75.14477 39.931...",271,"(LINESTRING (-75.1473109 39.93365889999998, -7..."
2786,3228.0,2.824500,165.066891,6,"POLYGON ((-75.15151 39.98508, -75.15151 39.983...",875,"(LINESTRING (-75.1505663 39.98718319999999, -7..."
3846,3160.0,0.540360,74.874407,6,"POLYGON ((-75.24808 39.90211, -75.24808 39.900...",171,"(LINESTRING (-75.250073 39.89994900000001, -75..."
3806,3158.0,0.476858,197.234122,7,"POLYGON ((-75.23236 39.91075, -75.23236 39.909...",151,"(LINESTRING (-75.233174 39.909074000000004, -7..."
3722,2943.0,2.157219,137.539194,5,"POLYGON ((-75.23236 39.91940, -75.23236 39.917...",733,"(LINESTRING (-75.2323653 39.9176228, -75.23300..."
...,...,...,...,...,...,...,...
1851,0.0,0.000000,0.000000,1,"POLYGON ((-75.01452 40.03171, -75.01452 40.029...",95,None
1852,0.0,0.000000,0.000000,1,"POLYGON ((-75.00778 40.03171, -75.00778 40.029...",1,None
1897,0.0,0.000000,0.000000,1,"POLYGON ((-75.01452 40.02998, -75.01452 40.028...",43,None


In [53]:
# Check which park is preferred per grid (the one with the best score)

grid_prefer = gridpark_score.iloc[gridpark_score.groupby('Grid_No')['score'].idxmax()]
grid_prefer = grid_prefer[['geometry_x','Parkroad_osmid','score','Grid_No','Park_No','geometry_y','population_score']]
grid_prefer = pd.merge(grid_prefer, PtR[['osmid','geometry_x']], left_on = 'Parkroad_osmid', right_on = 'osmid', how = 'left')
lines = list()
for i in range(len(grid_prefer)):
    if grid_prefer['geometry_x_y'].iloc[i] != None:
        lines.append(LineString([grid_prefer['geometry_y'].iloc[i].centroid, grid_prefer['geometry_x_y'].iloc[i]]))
    else: lines.append(None)
grid_prefer['lines'] = lines
gpd.GeoDataFrame(grid_prefer[['Parkroad_osmid','score','Grid_No','Park_No','population_score','lines']], 
                 geometry = 'lines', crs = 4326).to_file(r'C:\Users\bartb\Downloads\gridpark_preference.shp')
grid_prefer.sort_values('score')

,geometry_x_x,Parkroad_osmid,score,Grid_No,Park_No,geometry_y,population_score,osmid,geometry_x_y,lines
0,None,NaN,0.0,0,-1,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",0.0,NaN,None,None
1128,None,NaN,0.0,1128,-1,"POLYGON ((-75.08414 40.05415, -75.08414 40.052...",0.0,NaN,None,None
1152,None,NaN,0.0,1152,-1,"POLYGON ((-75.24359 40.05242, -75.24359 40.050...",0.0,NaN,None,None
1156,None,NaN,0.0,1156,-1,"POLYGON ((-75.21439 40.05242, -75.21439 40.050...",0.0,NaN,None,None
1170,None,NaN,0.0,1170,-1,"POLYGON ((-75.13579 40.05242, -75.13579 40.050...",0.0,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...
3109,"LINESTRING (-75.22225 39.96691, -75.22225 39.9...",1.097903e+08,995.0,3109,451,"POLYGON ((-75.22113 39.96780, -75.22113 39.966...",440785.0,1.097903e+08,POINT (-75.22225 39.96691),LINESTRING (-75.22225295789113 39.966937625158...
1808,"LINESTRING (-75.20430 40.03081, -75.20429 40.0...",1.097322e+08,995.0,1808,68,"POLYGON ((-75.20316 40.03171, -75.20316 40.029...",239795.0,1.097322e+08,POINT (-75.20430 40.03081),LINESTRING (-75.20428665220874 40.030844230137...
3410,"LINESTRING (-75.15267 39.94791, -75.15263 39.9...",3.590076e+09,995.0,3410,6,"POLYGON ((-75.15151 39.94879, -75.15151 39.947...",512425.0,3.590076e+09,POINT (-75.15267 39.94791),LINESTRING (-75.15263352337186 39.947926740848...
1428,"LINESTRING (-75.20206 40.04293, -75.20204 40.0...",5.884162e+09,998.0,1428,68,"POLYGON ((-75.20092 40.04379, -75.20092 40.042...",34930.0,5.884162e+09,POINT (-75.20206 40.04293),LINESTRING (-75.20204086399843 40.042927887393...


In [52]:
# 555 grids don't have access within the log adjusted 1000m threshold.
g = gridpark_score.iloc[gridpark_score.groupby('Grid_No')['score'].idxmax()]
g[g['score'] > 0]

,index,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,...,walk_area,log_infl_factor,raw_total_cost,grav_total_cost,parkcost_km2,gridpark_no,score,PoP2015_Number,geometry_y,population_score
0,117767,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,649.034,5,10,349,349,47.215,1.100560e+08,...,96266.908663,1.039125,696.249,670.033970,64.502099,10-349,330.0,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",41910.0
803,117769,"MULTILINESTRING ((-75.00807 40.12840, -75.0083...",normal way,695.146,5,15,349,349,56.061,1.100560e+08,...,96266.908663,1.039125,751.207,722.922702,69.593534,15-349,277.0,75,"POLYGON ((-75.00778 40.12832, -75.00778 40.126...",20775.0
962,117773,"MULTILINESTRING ((-75.00693 40.12429, -75.0061...",normal way,636.001,3,16,349,349,23.353,1.100560e+08,...,96266.908663,1.039125,659.354,634.528133,61.084062,16-349,365.0,51,"POLYGON ((-75.00329 40.12832, -75.00329 40.126...",18615.0
1140,79736,"MULTILINESTRING ((-75.00370 40.12489, -75.0039...",normal way,618.395,7,17,349,349,103.146,1.101586e+08,...,96382.211299,1.039320,721.541,694.243314,66.912706,17-349,306.0,41,"POLYGON ((-75.00104 40.12832, -75.00104 40.126...",12546.0
1818,117776,"MULTILINESTRING ((-75.00780 40.12481, -75.0082...",normal way,383.930,7,20,349,349,133.231,1.100560e+08,...,96266.908663,1.039125,517.161,497.688956,47.910977,20-349,502.0,155,"POLYGON ((-75.01003 40.12659, -75.01003 40.124...",77810.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5672,15265,"MULTILINESTRING ((-75.25583 39.89280, -75.2558...",normal way,31.873,2,3869,10,10,38.249,8.181536e+09,...,251348.233560,1.195570,70.122,58.651499,14.741951,3869-10,941.0,1,"POLYGON ((-75.25482 39.89346, -75.25482 39.891...",941.0
5674,15270,"MULTILINESTRING ((-75.25367 39.89423, -75.2534...",normal way,270.874,6,3870,10,10,151.206,1.103721e+08,...,209885.548510,1.166183,422.080,361.932885,75.964482,3870-10,638.0,23,"POLYGON ((-75.25257 39.89346, -75.25257 39.891...",14674.0
5675,15314,"MULTILINESTRING ((-75.25518 39.89278, -75.2550...",normal way,280.435,4,3873,10,10,375.117,8.181536e+09,...,211430.384059,1.167378,655.552,561.559083,118.730653,3873-10,438.0,7,"POLYGON ((-75.25257 39.89173, -75.25257 39.890...",3066.0
5676,15322,"MULTILINESTRING ((-75.25273 39.89373, -75.2520...",normal way,406.415,9,3874,10,10,305.591,1.103721e+08,...,209885.548510,1.166183,712.006,610.543939,128.144350,3874-10,389.0,13,"POLYGON ((-75.25033 39.89173, -75.25033 39.890...",5057.0


In [54]:
start_time = time.time()
# group the grid scores per park (which parks will be popular)
park_score = pd.DataFrame(gridpark_score.groupby('Park_No')['score'].sum().sort_values(ascending=False))
park_score['population_score (mln.)'] = gridpark_score.groupby('Park_No')['population_score'].sum()/1000000
park_score['parkscore_km2'] = gridpark_score.groupby('Park_No')['parkcost_km2'].sum()
park_score['Park access to pop.'] = gridpark_score.groupby('Park_No')['PoP2015_Number'].sum()
park_score['count'] = gridpark_score.groupby('Park_No')['score'].count()
park_score = park_score.join(parks_within_range[['geometry','OBJECTID']], how = 'outer')
park_score['park_area_m2'] = gpd.GeoSeries(park_score['geometry'], crs = 4326).to_crs(3043).area
park_score['park_score_km2_vs_total_park_size'] = park_score['parkscore_km2'] / (park_score['park_area_m2'] / 1000000)
park_score = park_score[park_score.index >= 0]
park_score['OBJECTID'] = park_score['OBJECTID'].astype(int)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
park_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Park_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score

time taken 0.0 mns


,score,population_score (mln.),parkscore_km2,Park access to pop.,count,geometry,OBJECTID,park_area_m2,park_score_km2_vs_total_park_size,lines
0,21325.0,2.085363,4769.315240,4305.0,39.0,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,4.192678e+06,1137.534419,"(LINESTRING (-75.2410148 40.0578267, -75.24021..."
1,1698.0,0.442554,182.571892,1832.0,6.0,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,5.470200e+04,3337.572304,"(LINESTRING (-75.2458592 39.91941099999998, -7..."
2,60918.0,24.266624,12237.835447,48337.0,111.0,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,7.283218e+06,1680.278566,"(LINESTRING (-75.250656 39.986672000000006, -7..."
3,2207.0,0.464605,13.439748,1019.0,5.0,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,4.811379e+03,2793.325380,"(LINESTRING (-75.150406 39.962371, -75.150315 ..."
4,1596.0,0.348272,57.770245,1589.0,7.0,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,1.069010e+04,5404.090479,(LINESTRING (-75.17361440000002 40.03491760000...
...,...,...,...,...,...,...,...,...,...,...
469,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,3.624449e+03,NaN,None
470,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,9.827071e+03,NaN,None
471,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,2.709308e+04,NaN,None
472,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,1.973472e+04,NaN,None


In [55]:
park_lines = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'lines']], 
                              geometry = 'lines', crs = 4326)
park_lines['Park_No'] = park_lines.index
park_lines.to_file(r'C:\Users\bartb\Downloads\park_entrance_routes.shp')

park_s = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'geometry']], 
                          geometry = 'geometry', crs = 4326)
park_s['Park_No'] = park_lines.index
park_s.to_file(r'C:\Users\bartb\Downloads\park_score_entrance.shp')

In [56]:
grid_lines = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','lines']], 
                              geometry = 'lines', crs = 4326)
grid_lines.to_file(r'C:\Users\bartb\Downloads\grid_entrance_routes.shp')

grid_s = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','geometry']], 
                          geometry = 'geometry', crs = 4326)
grid_s.to_file(r'C:\Users\bartb\Downloads\grid_score_entrance.shp')

In [36]:
print(round((time.time() - start) / 60,2),'mns')

14.68 mns
